In [75]:
#import libraries needed
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_rand_score, silhouette_score
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [84]:
#Read in dataset from umy computer

df = pd.read_csv(r'C:\Users\erinr\OneDrive\Documents\Capstone Project\COVID_Data.csv')

In [85]:
#remove NA values from df 
df.dropna(inplace=True)

In [86]:
df.head(5)


,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes,Country,Severity_Level
0,1,1,1,1,1,0,1,1,1,1,...,0,1,0,0,0,0,0,1,China,1
1,1,1,1,1,1,0,1,1,1,1,...,0,1,0,0,0,0,1,0,China,1
2,1,1,1,1,1,0,1,1,1,1,...,0,1,0,0,0,1,0,0,China,1
3,1,1,1,1,1,0,1,1,1,1,...,0,0,1,0,0,0,0,1,China,2
4,1,1,1,1,1,0,1,1,1,1,...,0,0,1,0,0,0,1,0,China,2


In [87]:
#drop the symptom columns from the dataset 
df=df.drop(['Fever','Tiredness','Dry-Cough','Difficulty-in-Breathing','Sore-Throat','None_Sympton','Pains','Nasal-Congestion','Runny-Nose','Diarrhea','Country','None_Experiencing','Contact_Dont-Know','Contact_No','Contact_Yes','Severity_Mild','Severity_Moderate','Severity_None'],axis=1) 

In [88]:
df.head(5)


,Age_0-9,Age_10-19,Age_20-24,Age_25-59,Age_60+,Gender_Female,Gender_Male,Gender_Transgender,Severity_Severe,Severity_Level
0,1,0,0,0,0,0,1,0,0,1
1,1,0,0,0,0,0,1,0,0,1
2,1,0,0,0,0,0,1,0,0,1
3,1,0,0,0,0,0,1,0,0,2
4,1,0,0,0,0,0,1,0,0,2


In [100]:
# Define lists of column names 
#indicators 1 will represent symptoms 
#indicators 2 will be used for modeling the symptoms 
Indicators1 = [ 'Age_10-19', 'Age_20-24', 'Age_25-59', 'Age_60+','Gender_Female'] 
target_columns = ['Severity_Severe'] 
indicators2 = Indicators1 + target_columns 

# Extract relevant columns from the DataFrame 
features = df[Indicators1] 
targets = df[target_columns] 

# Display the first few rows of the features and targets display
(features.head(), targets.head())



(   Age_10-19  Age_20-24  Age_25-59  Age_60+  Gender_Female
 0          0          0          0        0              0
 1          0          0          0        0              0
 2          0          0          0        0              0
 3          0          0          0        0              0
 4          0          0          0        0              0,
    Severity_Severe
 0                0
 1                0
 2                0
 3                0
 4                0)

In [102]:


#Define dependant and independant variables 
X = df[Indicators1]

y = df[target_columns]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# print the shapes of the train and test sets to verify the split
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (221760, 4), y_train shape: (221760, 1)
X_test shape: (95040, 4), y_test shape: (95040, 1)


In [93]:
# Fit the scaler on the training data and transform both the training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [94]:
# Print the shapes of the train and test sets to verify the split
print(f"X_train_scaled shape: {X_train_scaled.shape}, y_train shape: {y_train.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}, y_test shape: {y_test.shape}")

X_train_scaled shape: (221760, 7), y_train shape: (221760, 1)
X_test_scaled shape: (95040, 7), y_test shape: (95040, 1)


In [95]:
# Add a constant to the features matrix for the intercept term
X_train_const = sm.add_constant(X_train_scaled)

# Fit the ordinary least squares (OLS) model
model = sm.OLS(y_train, X_train_const)
results = model.fit()

# Print a summary of the regression results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:        Severity_Severe   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    -2.110
Date:                Sun, 23 Jun 2024   Prob (F-statistic):               1.00
Time:                        10:44:49   Log-Likelihood:            -1.2917e+05
No. Observations:              221760   AIC:                         2.584e+05
Df Residuals:                  221752   BIC:                         2.584e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2504      0.001    272.140      0.0

In [96]:
# Add a constant to the features matrix for the intercept term
X_train_const = sm.add_constant(X_train_scaled)

# Fit the ordinary least squares (OLS) model
model = sm.OLS(y_train, X_train_const)
results = model.fit()

# Print a summary of the regression results
print(results.summary())

# Get the names of the columns
# Insert 'Intercept' at the beginning for the constant term
column_names = list(X_train.columns)
column_names.insert(0, 'Intercept')  

# Print coefficients alongside column names
print("\nCoefficients:")
for col, coef in zip(column_names, results.params):
    print(f"{col}: {coef}")


                            OLS Regression Results                            
Dep. Variable:        Severity_Severe   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    -2.110
Date:                Sun, 23 Jun 2024   Prob (F-statistic):               1.00
Time:                        10:44:51   Log-Likelihood:            -1.2917e+05
No. Observations:              221760   AIC:                         2.584e+05
Df Residuals:                  221752   BIC:                         2.584e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2504      0.001    272.140      0.0

In [98]:
# Add a constant to the features matrix for the intercept term
X_train_const = sm.add_constant(X_train_scaled)

# Fit the logistic regression model
logit_model = sm.Logit(y_train, X_train_const)
results = logit_model.fit()

# Print a summary of the regression results
print(results.summary())

# Get the names of the columns 
# Insert 'Intercept' at the beginning for the constant term
column_names = list(X_train.columns)
column_names.insert(0, 'Intercept') 

# Print coefficients alongside column names
print("\nCoefficients:")
for col, coef in zip(column_names, results.params):
    print(f"{col}: {coef}")

# Calculate and print the odds ratios
odds_ratios = np.exp(results.params)
print("\nOdds Ratios:")
for col, odds in zip(column_names, odds_ratios):
    print(f"{col}: {odds}")

         Current function value: 0.562735
         Iterations: 35


C:\ProgramData\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                           Logit Regression Results                           
Dep. Variable:        Severity_Severe   No. Observations:               221760
Model:                          Logit   Df Residuals:                   221752
Method:                           MLE   Df Model:                            7
Date:                Sun, 23 Jun 2024   Pseudo R-squ.:               1.889e-05
Time:                        18:29:49   Log-Likelihood:            -1.2479e+05
converged:                      False   LL-Null:                   -1.2479e+05
Covariance Type:            nonrobust   LLR p-value:                    0.6946
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0966      0.005   -223.725      0.000      -1.106      -1.087
x1            -0.0094      0.006     -1.520      0.129      -0.022       0.003
x2            -0.0059      0.006     -0.952      0.3

In [103]:
# Define lists of column names 
#indicators 1 will represent symptoms 
#indicators 2 will be used for modeling the symptoms 
Indicators1 = [ 'Age_10-19', 'Age_20-24', 'Age_25-59', 'Age_60+',] 
target_columns = ['Severity_Severe'] 
indicators2 = Indicators1 + target_columns 

# Extract relevant columns from the DataFrame 
features = df[Indicators1] 
targets = df[target_columns] 

# Display the first few rows of the features and targets display
(features.head(), targets.head())

#Define dependant and independant variables 
X = df[Indicators1]

y = df[target_columns]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# print the shapes of the train and test sets to verify the split
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# Add a constant to the features matrix for the intercept term
X_train_const = sm.add_constant(X_train_scaled)

# Fit the logistic regression model
logit_model = sm.Logit(y_train, X_train_const)
results = logit_model.fit()

# Print a summary of the regression results
print(results.summary())

# Get the names of the columns
column_names = list(X_train.columns)
column_names.insert(0, 'Intercept')  # Insert 'Intercept' at the beginning for the constant term

# Print coefficients alongside column names
print("\nCoefficients:")
for col, coef in zip(column_names, results.params):
    print(f"{col}: {coef}")

# Calculate and print the odds ratios
odds_ratios = np.exp(results.params)
print("\nOdds Ratios:")
for col, odds in zip(column_names, odds_ratios):
    print(f"{col}: {odds}")


X_train shape: (221760, 4), y_train shape: (221760, 1)
X_test shape: (95040, 4), y_test shape: (95040, 1)
         Current function value: 0.562735
         Iterations: 35


C:\ProgramData\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                           Logit Regression Results                           
Dep. Variable:        Severity_Severe   No. Observations:               221760
Model:                          Logit   Df Residuals:                   221752
Method:                           MLE   Df Model:                            7
Date:                Sun, 23 Jun 2024   Pseudo R-squ.:               1.889e-05
Time:                        18:49:56   Log-Likelihood:            -1.2479e+05
converged:                      False   LL-Null:                   -1.2479e+05
Covariance Type:            nonrobust   LLR p-value:                    0.6946
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0966      0.005   -223.725      0.000      -1.106      -1.087
x1            -0.0094      0.006     -1.520      0.129      -0.022       0.003
x2            -0.0059      0.006     -0.952      0.3